In [1]:
from ns3gym import ns3env
from comet_ml import Experiment
import tqdm
import subprocess
from collections import deque
import numpy as np

from agents.ddpg.agent import Agent
from agents.teacher import Teacher

C:\ProgramData\Anaconda3\lib\site-packages\requests\__init__.py:80: RequestsDependencyWarning: urllib3 (1.24.1) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


### Basic constants and setting up environment

In [2]:
simTime = 10 # seconds
stepTime = 0.1  # seconds
history_length = 10

EPISODE_COUNT = 30
steps_per_ep = int(simTime/stepTime)

SCRIPT_RUNNING = False

script_exec_command = f'../../waf --run "linear-mesh --simTime={simTime} --envStepTime={stepTime} --historyLength={history_length}"'
print("Steps per episode:", steps_per_ep)

Steps per episode: 100


In [3]:
port = 5555
seed = 0

simArgs = {"--simTime": simTime,
           "--testArg": 123,
           "--nodeNum": 5,
           "--distance": 500}
debug = False

subprocess.Popen(['bash', '-c', script_exec_command])
SCRIPT_RUNNING = True

env = ns3env.Ns3Env(port=port, stepTime=stepTime, startSim=0, simSeed=seed, simArgs=simArgs, debug=debug)

Waiting for simulation script to connect on port: tcp://localhost:5555
Please start proper ns-3 simulation script using ./waf --run "..."


In [4]:
env.reset()
ob_space = env.observation_space
ac_space = env.action_space

print("Observation space shape:", ob_space)
print("Action space shape:", ac_space)

assert ob_space is not None

Observation space shape: Box(10,)
Action space shape: Box(1,)


### Creating and training agent

In [5]:
if not SCRIPT_RUNNING:
    subprocess.Popen(['bash', '-c', script_exec_command])
    SCRIPT_RUNNING = True
    
agent = Agent(history_length, 1)
teacher = Teacher(agent, env, 1)
teacher.train(EPISODE_COUNT, simTime, stepTime, "Inp: window Mb sent", "Rew: normalized speed", "Warmup", "Reward - 1/2")

SCRIPT_RUNNING = False

COMET INFO: Experiment is live on comet.ml https://www.comet.ml/wwydmanski/rl-in-wifi/87051e08d6634507b8db815038d4f49d

 67%|█████████████████████████████████████████▌                    | 67/100 [00:49<00:24,  1.35it/s, mb_sent=245.33 Mb]

------- STARTED TRAINING -------


100%|█████████████████████████████████████████████████████████████| 100/100 [01:12<00:00,  1.37it/s, mb_sent=370.14 Mb]


Sent 370.14 Mb/s.	Mean speed: 37.01 Mb/s	Episode 1/30 finished

Waiting for simulation script to connect on port: tcp://localhost:5555
Please start proper ns-3 simulation script using ./waf --run "..."


100%|█████████████████████████████████████████████████████████████| 100/100 [01:09<00:00,  1.43it/s, mb_sent=375.97 Mb]


...

Sent 387.70 Mb/s.	Mean speed: 38.77 Mb/s	Episode 29/30 finished

Waiting for simulation script to connect on port: tcp://localhost:5555
Please start proper ns-3 simulation script using ./waf --run "..."


100%|█████████████████████████████████████████████████████████████| 100/100 [01:08<00:00,  1.47it/s, mb_sent=388.31 Mb]


Sent 388.31 Mb/s.	Mean speed: 38.83 Mb/s	Episode 30/30 finished



COMET INFO: ----------------------------
COMET INFO: Comet.ml Experiment Summary:
COMET INFO:   Metrics:
COMET INFO:                     Chosen CW: 39.889043730598296
COMET INFO:             Cumulative reward: 1.7740478431805968
COMET INFO:                Megabytes sent: 388.3053509999999
COMET INFO:                 Per-ep reward: 1.7740478431805968
COMET INFO:          Round megabytes sent: 3.786438
COMET INFO:                  Round reward: 0.0048583983443677425
COMET INFO:                         Speed: 38.83053509999999
COMET INFO:                    actor_loss: 0.02593047
COMET INFO:                   critic_loss: 0.009426987
COMET INFO:                          loss: 0.02593046985566616
COMET INFO:         sys.gpu.0.free_memory: 8315416576
COMET INFO:     sys.gpu.0.gpu_utilization: 5
COMET INFO:        sys.gpu.0.total_memory: 11811160064
COMET INFO:         sys.gpu.0.used_memory: 3495743488
COMET INFO: ----------------------------
COMET INFO: Uploading stats to Comet before progr

Training finished.
